In [29]:
#Steganography project
#Zachary Miles
#This notebook was designed to run in Python 3, if the notebook is to be run in 
#Python 2, four small changes must be made, two in the stegWrite function and two in the stegRead
#function regarding fixing the operator for integer division. 
#Each line that needs a fix for Python 2 is marked with a comment labeled 'WARNING:'
from PIL import Image
import random
imag = "Enter a filepath for an image to hide information in here"
sour = "Enter a filepath for a text file containing a message here"

In [25]:
#Returns a string containing all text in an input file
#Requires a filepath for an argument
def readFile(path):
    fileString = ""
    for line in open(path):
        line = line.replace("\n", " ")
        fileString += line
    return fileString

In [26]:
#Generates an integer to seed a random number generator from a string input
def createSeed(stri):
    i = 1
    for c in stri:
        i = (i + ord(c))*3
    return i
createSeed("Seedtest")

883446

In [27]:
#Function to hide information within an image
#sourcepath should be a path to a textfile with the information to hide
#The source information should be 8 KB or less for the function to work properly
#imagepath should be a path to an image to use for hiding information
#genSeed should be a string paraphrase which will be used to seed the random number generator for the function
#destPath defines where the resulting image with the hidden data will be saved at
#By default, this function reads information from a file, but removing the readFile function from the first line and
#assigning sourcepath directly to stri will allow you to pass a string directly to the function to test more easily
def stegWrite(sourcepath, imagepath, genSeed, destPath):
    stri = readFile(sourcepath) #Removing the read file function here and simply assigning sourcepath to stri will allow for a message to be directly passed to the function
    bina = ""
    for i in stri:
        ch = format(ord(i), 'b') #convert each character in input to binary ascii value
        if len(ch) < 7:
            ch = "0" + ch
        bina = bina + ch
    pict = Image.open(imagepath)
    binsize = len(bina) #record number of bits of input text
    print(pict.format, pict.size, binsize)
    random.seed(createSeed(genSeed))
    si = bin(binsize)[2:] #convert number of bits to binary
    while len(si) < 16:
        si = "0" + si #add leading zero to number of bits in binary to ensure there is consistently 16 bits to read
    px = pict.load()
    matrix = [[0 for h in range(pict.size[1])] for w in range(pict.size[0])] #This matrix will be used to keep track of duplicate pixels
    #add number of bits to random pixels in image
    prog = 0
    while prog < 16:
        loc = random.randint(0, 1000000000000000000000000000000000000000000000000000000000000000000000000)
        i = loc//pict.size[1]%pict.size[0] #WARNING: If this program is run on Python 2 the division operator must be changed to '/' to do integer division
        j = loc%pict.size[1]
        p = px[i, j]
        n = 0
        if si[prog] == '0':
            n = p[0] & 254 #highest possible RGB value is 255/11111111, so & 254/11111110 should only change the last bit of any possible value
        else:
            n = p[0] | 1
        k = matrix[i][j]
        #The value of k tells us how many times the chosen pixel has been used before
        if k == 0:
            px[i, j] = (n, p[1], p[2]) #The first time a pixel is chosen, the bit is assigned to the R value
            matrix[i][j] = 1
            prog = prog + 1
        elif k == 1:
            px[i, j] = (p[0], n, p[2]) #The second time a pixel is chosen, the bit is assinged to the G value
            matrix[i][j] = 2
            prog = prog + 1
        elif k == 2:
            px[i, j] = (p[0], p[1], n) #The third time a pixel is chosen, the bit is assigned to the B value
            matrix[i][j] = 3
            prog = prog + 1    
    #add message to random pixels in image
    prog = 0
    while prog < binsize:
        loc = random.randint(0, 1000000000000000000000000000000000000000000000000000000000000000000000000)
        i = loc//pict.size[1]%pict.size[0] #WARNING: If this program is run on Python 2 the division operator must be changed to '/' to do integer division
        j = loc%pict.size[1]
        p = px[i, j]
        n = 0
        if bina[prog] == '0':
            n = p[0] & 254 #highest possible RGB value is 255/11111111, so & 254/11111110 should only change the last bit of any possible value
        else:
            n = p[0] | 1
        k = matrix[i][j]
        #The value of k tells us how many times the chosen pixel has been used before
        if k == 0:
            px[i, j] = (n, p[1], p[2]) #The first time a pixel is chosen, the bit is assigned to the R value
            matrix[i][j] = 1
            prog = prog + 1
        elif k == 1:
            px[i, j] = (p[0], n, p[2]) #The second time a pixel is chosen, the bit is assinged to the G value
            matrix[i][j] = 2
            prog = prog + 1
        elif k == 2:
            px[i, j] = (p[0], p[1], n) #The third time a pixel is chosen, the bit is assigned to the B value
            matrix[i][j] = 3
            prog = prog + 1
    pict.save(destPath)
    print("Saved picture to " + destPath)

In [30]:
stegWrite(sour, imag, "This is a seed", "secret.png")

PNG (1294, 1286) 6650
Saved picture to secret.png


In [20]:
#Function to read information from an image
#imagePath should be a path to the image containing the information to read
#genSeed should be a string paraphrase which will be used to seed the random number generator for the function
def stegRead(imagePath, genSeed):
    pict = Image.open(imagePath)
    random.seed(createSeed(genSeed))
    px = pict.load()
    numStr = ""
    matrix = [[0 for h in range(pict.size[1])] for w in range(pict.size[0])] #matrix to keep track of duplicate pixels
    x = 0
    #Always retrieves the first 16 bits, this tells us how large the message size is
    while x < 16:
        loc = random.randint(0, 1000000000000000000000000000000000000000000000000000000000000000000000000)
        i = loc//pict.size[1]%pict.size[0] #WARNING: If this program is run on Python 2 the division operator must be changed to '/' to do integer division
        j = loc%pict.size[1]
        p = px[i, j]
        k = matrix[i][j]
        #The value of k tells us how many times the chosen pixel has been used before
        if k == 0:
            numStr += str(p[0] & 1) #The first time a pixel is chosen, we read the bit from the R value
            matrix[i][j] = 1
            x = x + 1
        elif k == 1:
            numStr += str(p[1] & 1) #The second time a pixel is chosen, we read the bit from the G value
            matrix[i][j] = 2
            x = x + 1
        elif k == 2:
            numStr += str(p[2] & 1) #The third time a pixel is chosen, we read the bit from the B value
            matrix[i][j] = 3
            x = x + 1
    num = int(numStr, 2)
    x = 0
    binMessage = ""
    #Using the message size, we can now read out and build the message from the picture
    while x < num:
        loc = random.randint(0, 1000000000000000000000000000000000000000000000000000000000000000000000000)
        i = loc//pict.size[1]%pict.size[0] #WARNING: If this program is run on Python 2 the division operator must be changed to '/' to do integer division
        j = loc%pict.size[1]
        p = px[i, j]
        k = matrix[i][j]
        #The value of k tells us how many times the chosen pixel has been used before
        if k == 0:
            binMessage += str(p[0] & 1) #The first time a pixel is chosen, we read the bit from the R value
            matrix[i][j] = 1
            x = x + 1
        elif k == 1:
            binMessage += str(p[1] & 1) #The second time a pixel is chosen, we read the bit from the G value
            matrix[i][j] = 2
            x = x + 1
        elif k == 2:
            binMessage += str(p[2] & 1) #The third time a pixel is chosen, we read the bit from the B value
            matrix[i][j] = 3
            x = x + 1
    #print(binMessage)
    message = ""
    #Builds a message by turning the binary data to the acsii int value, then converting the ascii int value to a character
    #and appending it to message
    for y in range(0, len(binMessage), 7):
        asciiVal = int(binMessage[y:y+7], 2)
        message += chr(asciiVal)
        #print(chr(asciiVal))
    #print(message)
    return message

In [31]:
stegRead("secret.png", "This is a seed")

"Vader: You are beaten. It is useless to resist. Don't let yourself be destroyed as Obi-Wan did. Vader: There is no escape. Don't make me destroy you. Luke, you do not yet realize your importance. You've only begun to discover your power. Join me, and I will complete your training. With our combined strength, we can end this destructive conflict and bring order to the galaxy! Luke: I'll never join you! Vader: If only you knew the power of the dark side. Obi-Wan never told you what happened to your father. Luke: He told me enough. He told me you killed him. Vader: No. I am your father. Luke: No. No That's not true! That's impossible! Vader: Search your feelings you know it to be true! Luke: NO!!! NO!!! Vader: Luke, you can destroy the emperor. He has foreseen this. It is your destiny. Join me, and together, we can rule the galaxy as father and son! Come with me, it is the only way. [Luke lets go of the projection and falls into the shaft]"